### Biodiversity Python Project by Albert Cort Banke
_____________________________________________________________________________________________________________________________________________

Project scope

***Goal***: To draw insights from the biodiversity data included and illustrate interesting features by answering questions with visualizations and summary statistics

***Data***: There are two datasets. One is data about species and the other is data about observations of different species in national parks 

***Analysis***: We use exploratory data analysis and hypothesis testing to assess the data and determine associations in the data.

***Questions***: In what ways do species represent biodiversity? What parks have the most biodiversity? Why?

_____________________________________________________________________________________________________________________________________________

**1. Preparation**

Import the relevant libraries (A) and import and read the data (B)

In [5]:
# (A)
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import seaborn as sns

In [6]:
# (B)
species = pd.read_csv('species_info.csv')

obs = pd.read_csv('observations.csv')

# We print the first few observations to check the data has been imported correctly

print(species.head())
print(obs.head())

# The datasets look correct at first glance, however, in the species dataset the conservation_status column is apparently blank.
# Moreover, the common_names column contains multiple names for the scientific_name. This is important to note

  category                scientific_name  \
0   Mammal  Clethrionomys gapperi gapperi   
1   Mammal                      Bos bison   
2   Mammal                     Bos taurus   
3   Mammal                     Ovis aries   
4   Mammal                 Cervus elaphus   

                                        common_names conservation_status  
0                           Gapper's Red-Backed Vole                 NaN  
1                              American Bison, Bison                 NaN  
2  Aurochs, Aurochs, Domestic Cattle (Feral), Dom...                 NaN  
3  Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)                 NaN  
4                                      Wapiti Or Elk                 NaN  
            scientific_name                            park_name  observations
0        Vicia benghalensis  Great Smoky Mountains National Park            68
1            Neovison vison  Great Smoky Mountains National Park            77
2         Prunus subcordata               

In [7]:
# Check for missing values in the datasets

print(species.info())

# The conservation-status has an overload of missing entries. The other columns in the species dataset are complete 

print(obs.info())

# The observation dataset is complete. No missing values are present 

# From the two info() headers we see that most of the data has an object datatype. One is an int64, or, a quantative variable. 
# This means most of the data is strings or text
# This is important to note beacause it affects which visualisations and statistics are wise to use 

print(species.conservation_status.unique())

# The unique values for this column is 'Species of Concern' 'Endangered' 'Threatened' 'In Recovery' and 'nan'. 
# The 'nan' value could represent a missing value, but perhaps it could represent a condition status that is not on the current axis. 
# This could be a an indication 'Good' or 'Thriving' for example. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB
None
[nan 'Species of Concern' 'Endangered' 'Threatened' 'In Recovery']


2. Data transformation

Prepare (C) the dataset and review (D) it for the exploratory data analysis

In [8]:
# (C) Prepare the dataset by addressing their originality

print(obs.scientific_name.is_unique) # Prints false - this means we have multiple instances of the same scientific_name
print(obs.park_name.is_unique) # Prints false - this means we have multiple instances of the same park_name

# We can now assume that we have species being observed across different parks with different observation numbers
# A simple but strong assumption. Important for merging the two datasets
# Were gonna pivot the column values in the park_name column into columns 
# Then were going to create a total observations column

obs = obs.pivot_table(index = 'scientific_name', columns = 'park_name', values = 'observations', aggfunc = np.sum)

obs = pd.DataFrame(obs.to_records()) #flattening the dataframe

obs['total_observations'] = obs['Bryce National Park'] + obs['Great Smoky Mountains National Park'] + obs['Yellowstone National Park'] + obs['Yosemite National Park']

# drop any duplicates in the species dataframe

species.drop_duplicates('scientific_name', inplace = True)

print(species.scientific_name.is_unique) # prints true - this means we have unique values for the scientific_name column

# Preparing the dataset by merging it. We can do so beacuse of the scientific_name column existing in both datasets and its uniqueness. 

biodiversity = pd.merge(
    species,
    obs,
    on = 'scientific_name',
    how = 'inner'
)

# Check to see if the merge is correct

print(biodiversity.head())

print(biodiversity.scientific_name.is_unique)

# Looks good. We see the columns and now we check the vitals
 

False
False
True
  category                scientific_name  \
0   Mammal  Clethrionomys gapperi gapperi   
1   Mammal                      Bos bison   
2   Mammal                     Bos taurus   
3   Mammal                     Ovis aries   
4   Mammal                 Cervus elaphus   

                                        common_names conservation_status  \
0                           Gapper's Red-Backed Vole                 NaN   
1                              American Bison, Bison                 NaN   
2  Aurochs, Aurochs, Domestic Cattle (Feral), Dom...                 NaN   
3  Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)                 NaN   
4                                      Wapiti Or Elk                 NaN   

   Bryce National Park  Great Smoky Mountains National Park  \
0                  130                                   98   
1                   68                                   77   
2                   99                                   56   
3  

In [ ]:
# (D) Review the dataset by looking at its vitals and the summary statistics 

# First and foremost, we need to do something about the NaN's (missing values). Most conservation_status classifications are empty.

# To check how we can map the missing we should divide the dataset into the respective category of species (mammels, birds etc.). Then we can check which values are present and their
# respective conservation status. From these, we can create a template for how many observations each classification should have. 



#print(obs.nlargest(5, 'observations'))

# (D) To review the dataset we look at some overall summary statistics. Important note: The data is mainly categorical. 
# This implicates the quantative summary statistics. 
